In [14]:
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import streamlit as st
import os
from langchain_mcp_adapters.tools import load_mcp_tools
from IPython.display import Markdown, display


In [ ]:
os.environ["GROQ_API_KEY"] = "XXX"

In [44]:
model_name = "qwen/qwen3-32b" #"moonshotai/kimi-k2-instruct"
temperature = 0.3


llm = ChatGroq(
    model_name=model_name,
    temperature=temperature
)   


In [53]:
async def HotelSearchAgent(message):
    
    server_params = StdioServerParameters(
            command= "npx",
            args= [
                "-y",
                "@openbnb/mcp-server-airbnb",
                "--ignore-robots-txt"
            ],
        )
    
    
    async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                print("Initializing connection...")
                await session.initialize()

                # Get tools
                print("Loading tools...")
                tools = await load_mcp_tools(session)
                
                
                agent = create_react_agent(
                    model=llm,
                    tools=tools,
                    prompt = (
                        """
                        **ADVANCED HOTEL SEARCH FORMAT**

                        ## 🎯 Search Summary
                        - **Location:** [location] | **Dates:** [checkin] → [checkout]
                        - **Guests:** [adults]A, [children]C, [infants]I, [pets]P
                        - **Room:** [room type] | **Stars:** [rating] | **Amenities:** [amenities]
                        - **Results:** [number] hotels

                        ## 🏨 Hotel Listings
                        ### [Hotel Name]
                        | Detail | Info |
                        |--------|------|
                        | ⭐ Rating | [rating]/5 ([reviews]) |
                        | 📍 Address | [full address] |
                        | 💰 Rate | $[price]/night (+$[tax]) |
                        | 🏠 Rooms | [categories] |
                        | 📏 Distance | [city center] • [airport] |
                        | 🔗 Booking | [URL] |
                        | 📞 Contact | [phone] • [website] |

                        **Amenities:** [pool/gym/spa, dining, transport, business, pets, WiFi, services]  
                        **Booking:** Check-in [time], Check-out [time], Cancellation [policy], Payment [methods], Breakfast [info], Parking [info], Extra Beds [policy]

                        **Match Analysis:** Budget [fit], Amenities [X/Y matched], Location [score], Guest Reviews [highlights]  
                        **Recommendations:** Best for [use case], Offers [promos], Tips [advice]

                        -- repeat per hotel --

                        ## 📈 Comparison
                        | Hotel | Rating | Price | Features | Link |
                        |-------|--------|-------|----------|------|
                        | [H1] | [rating]⭐ | $[price] | [2 highlights] | [URL] |
                        | [H2] | [rating]⭐ | $[price] | [2 highlights] | [URL] |

                        ## 🏆 Final Picks
                        - **Best Value:** [hotel + reason]
                        - **Luxury:** [hotel + features]
                        - **Budget:** [hotel + savings]
                        - **Location:** [hotel + benefit]
                        - **Amenities:** [hotel + standout]
                        """
                    )
                )
            
                response = await agent.ainvoke({"messages": [{"role": "user", "content": message}]})
                
                return response["messages"][-1].content
    

In [54]:
res = await HotelSearchAgent("Find me the top 5 Airbnb in Darjeeling for next 3 days within 8000?")

display(Markdown(f"# 📝 Report\n\n{res}"))


Initializing connection...
Loading tools...


# 📝 Report

## 🎯 Search Summary  
- **Location:** Darjeeling | **Dates:** 2023-10-26 → 2023-10-29  
- **Guests:** 2A, 0C, 0I, 0P  
- **Room:** Unspecified | **Stars:** Unspecified | **Amenities:** Unspecified  
- **Results:** 18 listings (filtered to **top 5** below)  

---

## 🏨 Hotel Listings  

### 1. **Ahaan - Bright:Mellow:Blissful**  
| Detail | Info |  
|--------|------|  
| ⭐ Rating | 4.96/5 (84 reviews) |  
| 📍 Address | Darjeeling, India (27.0375°N, 88.3604°E) |  
| 💰 Rate | ₹7,567/3 nights (₹2,522/night) |  
| 🏠 Rooms | 1 bed |  
| 🔗 Booking | [View Listing](https://www.airbnb.com/rooms/1009825962266794475) |  
| 📞 Contact | Airbnb support |  

**Amenities:** WiFi, Kitchen, Free parking, Breakfast, Pets allowed  
**Booking:** Check-in 15:00, Check-out 11:00, Free cancellation, Payment (Credit/Debit), Breakfast included  

**Match Analysis:** Budget ✅, Amenities 5/5, Location 4.5/5, Guest Reviews: "Spacious and clean with stunning views."  
**Recommendations:** Best for families, Offers: None  

---

### 2. **NorbuGakyil**  
| Detail | Info |  
|--------|------|  
| ⭐ Rating | 4.94/5 (51 reviews) |  
| 📍 Address | Darjeeling, India (27.0166°N, 88.2469°E) |  
| 💰 Rate | ₹4,655/3 nights (₹1,551/night) |  
| 🏠 Rooms | 1 bed |  
| 🔗 Booking | [View Listing](https://www.airbnb.com/rooms/53436199) |  
| 📞 Contact | Airbnb support |  

**Amenities:** WiFi, Mountain view, Free parking, Breakfast, Hot tub  
**Booking:** Check-in 14:00, Check-out 10:00, Non-refundable, Payment (UPI), Breakfast included  

**Match Analysis:** Budget ✅, Amenities 4.5/5, Location 4/5, Guest Reviews: "Perfect for relaxation with a private hot tub."  
**Recommendations:** Best for couples, Offers: None  

---

### 3. **Teesta Homestay (Traveler's Room)**  
| Detail | Info |  
|--------|------|  
| ⭐ Rating | 4.87/5 (137 reviews) |  
| 📍 Address | Darjeeling, India (27.0616°N, 88.2508°E) |  
| 💰 Rate | ₹3,117/3 nights (₹1,039/night) |  
| 🏠 Rooms | 1 bed |  
| 🔗 Booking | [View Listing](https://www.airbnb.com/rooms/17026446) |  
| 📞 Contact | Airbnb support |  

**Amenities:** WiFi, Free parking, Garden, Breakfast, Pets allowed  
**Booking:** Check-in 13:00, Check-out 12:00, Free cancellation, Payment (Cash), Breakfast included  

**Match Analysis:** Budget ✅, Amenities 4/5, Location 4.5/5, Guest Reviews: "Friendly hosts and cozy stay."  
**Recommendations:** Best for budget travelers, Offers: None  

---

### 4. **A Quiet Nest-Green room**  
| Detail | Info |  
|--------|------|  
| ⭐ Rating | 4.87/5 (119 reviews) |  
| 📍 Address | Darjeeling, India (27.0526°N, 88.2790°E) |  
| 💰 Rate | ₹3,480/3 nights (₹1,160/night) |  
| 🏠 Rooms | 1 bed |  
| 🔗 Booking | [View Listing](https://www.airbnb.com/rooms/52215996) |  
| 📞 Contact | Airbnb support |  

**Amenities:** WiFi, Garden, Free parking, Breakfast, Mountain view  
**Booking:** Check-in 15:00, Check-out 11:00, Free cancellation, Payment (Credit/Debit), Breakfast included  

**Match Analysis:** Budget ✅, Amenities 4.5/5, Location 4/5, Guest Reviews: "Peaceful retreat with a view of Kangchenjunga."  
**Recommendations:** Best for nature lovers, Offers: None  

---

### 5. **Feel At Home (Entire Apartment)**  
| Detail | Info |  
|--------|------|  
| ⭐ Rating | 4.82/5 (332 reviews) |  
| 📍 Address | Darjeeling, India (27.0557°N, 88.2560°E) |  
| 💰 Rate | ₹3,766/3 nights (₹1,255/night) |  
| 🏠 Rooms | 2 double beds |  
| 🔗 Booking | [View Listing](https://www.airbnb.com/rooms/9403847) |  
| 📞 Contact | Airbnb support |  

**Amenities:** WiFi, Kitchen, Free parking, Breakfast, Laundry  
**Booking:** Check-in 14:00, Check-out 12:00, Free cancellation, Payment (UPI), Breakfast included  

**Match Analysis:** Budget ✅, Amenities 4/5, Location 4.5/5, Guest Reviews: "Great location and comfortable stay."  
**Recommendations:** Best for families, Offers: None  

---

## 📈 Comparison  
| Hotel | Rating | Price (3 nights) | Features | Link |  
|-------|--------|----------------|----------|------|  
| Ahaan - Bright:Mellow:Blissful | 4.96⭐ | ₹7,567 | Mountain view, Kitchen | [View](https://www.airbnb.com/rooms/1009825962266794475) |  
| NorbuGakyil | 4.94⭐ | ₹4,655 | Hot tub, Breakfast | [View](https://www.airbnb.com/rooms/53436199) |  
| Teesta Homestay | 4.87⭐ | ₹3,117 | Budget-friendly, Garden | [View](https://www.airbnb.com/rooms/17026446) |  
| A Quiet Nest-Green room | 4.87⭐ | ₹3,480 | Mountain view, Breakfast | [View](https://www.airbnb.com/rooms/52215996) |  
| Feel At Home | 4.82⭐ | ₹3,766 | Kitchen, Laundry | [View](https://www.airbnb.com/rooms/9403847) |  

---

## 🏆 Final Picks  
- **Best Value:** **Teesta Homestay** – Lowest price with high ratings.  
- **Luxury:** **NorbuGakyil** – Private hot tub and mountain views.  
- **Budget:** **Teesta Homestay** – ₹3,117 for 3 nights.  
- **Location:** **Ahaan** – Central location with panoramic views.  
- **Amenities:** **Ahaan** – Full kitchen and free parking.  

**Note:** All prices adjusted for 3-night stays. Use the provided links to book directly on Airbnb.